In [1]:
!pip install transformers
!pip install wikipedia
!pip install arabert

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=71fb694bf79d24db0fe421f93996fbfdf8960e5f7a8fab58609e762cac644a52
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 9.0 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186456 sha256=d4f5755b0d43900825e35606000665505ac95b87b524d4013898aba4a9b53572
  Stored in directory: /root/.cache/pip/wheels/10/f0/fd/4813b1177405693e8da9cdea839f0fb64fde161380e058c827
Successfully built emoji


In [2]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from transformers import BertTokenizer, AutoModelForSeq2SeqLM, pipeline
from arabert.preprocess import ArabertPreprocessor
import wikipedia

# Load the Excel sheet
df = pd.read_excel('/content/stock_data.xlsx')

# Prepare the data for the model
X = df[['final price today', 'change today']].values
y = df['future_price'].values

# Create a pipeline with a scaler and a linear regression model
model_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

# Train the pipeline
model_pipeline.fit(X, y)

# Initialize the summary generation pipeline
model_name = "malmarjeh/mbert2mbert-arabic-text-summarization"
preprocessor = ArabertPreprocessor(model_name="")
tokenizer = BertTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
summarizer = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

# Function to get stock information, predict future price, and generate summary
def get_stock_info(stock_name):
    # Check if the name is in the list
    stock_info = df[df['name'] == stock_name]
    if stock_info.empty:
        return {"Error": "Stock not found"}

    # Set Wikipedia language to Arabic
    wikipedia.set_lang("ar")
    try:
        # Search with the stock name
        wiki_summary = wikipedia.summary("شركة " + stock_name + " السعودية", sentences=5)
    except wikipedia.exceptions.DisambiguationError as e:
        wiki_summary = f"Multiple entries found for {stock_name}: {e.options}"
    except wikipedia.exceptions.PageError:
        wiki_summary = "Wikipedia page not found for this company."

    # Setting the data to make the prediction and summary
    info = stock_info.iloc[0]
    current_price = info['final price today']
    change_today = info['change today']

    # Making the prediction
    future_price = model_pipeline.predict([[current_price, change_today]])[0]
    summary_prompt = f"سهم شركة {stock_name} سعره الحالي {current_price} اليوم مع تغيير قدره {change_today}. ويتوقع ان يصبح سعره {future_price} والجدير بالذكر ان "
    text = preprocessor.preprocess(wiki_summary)

    # Building the summary
    summary = summarizer(text,
                         pad_token_id=tokenizer.eos_token_id,
                         num_beams=3,
                         repetition_penalty=3.0,
                         max_length=300,
                         length_penalty=1.0,
                         no_repeat_ngram_size=3)[0]['generated_text']

    return {
        "اخر قيمة لليوم": current_price,
        "التغير": change_today,
        "السعر المتوقع": future_price,
        "ملخص": summary_prompt + summary,
        "مقال ويكيبيديا الكامل": wiki_summary
    }

# Example usage
stock_name = "أرامكو السعودية"
result = get_stock_info(stock_name)
print(result)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/534 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.73k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/828M [00:00<?, ?B/s]

The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']
Setting `pad_token_id` to `eos_token_id`:102 for open-end generation.


{'اخر قيمة لليوم': 27.2, 'التغير': -0.05, 'السعر المتوقع': 27.34201155455955, 'ملخص': 'سهم شركة أرامكو السعودية سعره الحالي 27.2 اليوم مع تغيير قدره -0.05. ويتوقع ان يصبح سعره 27.34201155455955 والجدير بالذكر ان أرامكو السعودية تحقق أرباحا سنويا في تاريخها', 'مقال ويكيبيديا الكامل': 'أرامكو السعودية (بالإنجليزية: Saudi Aramco)\u200f واسمها رسميًا شركة الزيت العربية السعودية (بالإنجليزية: Saudi Arabian Oil Co.)\u200f واختصارًا: أرامكو (بالإنجليزية: Aramco)\u200f، هي شركة سعودية وطنية تعمل في مجالات النفط والغاز الطبيعي والبتروكيماويات والأعمال المتعلقة بها من تنقيب وإنتاج وتكرير وتوزيع وشحن وتسويق، وهي شركة عالمية متكاملة تم تأميمها عام 1988م، يقع مقرها الرئيسي في الظهران. وتعد أكبر شركة في العالم من حيث القيمة السوقية حيث بلغت قيمتها السوقية 781 مليار في عام 2006م، و7 تريليون دولار في عام 2010 طبقاً لتقدير صحيفة فاينانشال تايمز. فيما رجحت مجلة اكسبلوريشن قيمة أرامكو السوقية في عام 2015 بحوالي 10 تريليون دولار، وبلغ إجمالي أرباح أرامكو 111 مليار دولار خلال عام 2018م، أي ما يعادل أرباح ش